# 04 — Batch Inference with QLoRA Adapter
Load a HuggingFace model with QLoRA adapter and run batch inference on a dataset.

**Note:** vLLM has limited LoRA support. We'll merge the adapter first for vLLM, or use transformers directly.

In [1]:
# Cell 1: Imports
import json
from pathlib import Path
from typing import List, Dict
import polars as pl
from tqdm.auto import tqdm

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import sys
import os

# Optional: for vLLM
try:
    # Add vLLM installation path
    vllm_path = "/Users/a.darryl/Desktop/Y4S2/finetuning/.venv/bin/vllm"
    sys.path.append(os.path.dirname(vllm_path))
    from vllm import LLM, SamplingParams
    VLLM_AVAILABLE = True
except ImportError:
    VLLM_AVAILABLE = False
    print("vLLM not installed. Will use transformers for inference.")

Skipping import of cpp extensions due to incompatible torch version 2.10.0 for torchao version 0.15.0             Please see https://github.com/pytorch/ao/issues/2919 for more info
W0206 20:44:31.637000 74924 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


INFO 02-06 20:44:32 [importing.py:68] Triton not installed or not compatible; certain GPU-related functions will not be available.


In [2]:
# Cell 2: Config
project_root = Path('..').resolve()
dataset_path = project_root / 'data' / 'poem_condense.csv'
output_path = project_root / 'data' / 'poem_condense_annotated.jsonl'

# Model configuration
adapter_model_id = "prettyvampire/poemma"  # HF model with QLoRA adapter
base_model_name = None  # Will auto-detect from adapter config

# Inference settings
use_vllm = True  # Set True to merge adapter and use vLLM (faster for large batches)
batch_size = 8  # For transformers inference
max_new_tokens = 256
temperature = 0.7
top_p = 0.9

# Text column to process
text_column = "Verse"  # Column name in your CSV
output_column = "analysis"  # Column name for output

In [3]:
# Cell 2b: Test mode for local macOS testing
import platform

# Toggle this for local testing
TEST_MODE = True  # Set False for full GPU cluster run

if TEST_MODE:
    print("⚠️  TEST MODE ENABLED - Using local settings")
    
    # Detect device
    if platform.system() == "Darwin":  # macOS
        if torch.backends.mps.is_available():
            device = "mps"  # Apple Silicon GPU
            print("✓ Using Apple Silicon GPU (MPS)")
        else:
            device = "cpu"
            print("✓ Using CPU (Intel Mac)")
    else:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"✓ Using {device}")
    
    # Override settings for testing
    batch_size = 2  # Smaller for CPU/MPS
    max_new_tokens = 50  # Shorter generations
    use_vllm = True
    
    # Test on subset only
    TEST_SUBSET_SIZE = 5  # Process only first N rows
    print(f"📊 Will process only {TEST_SUBSET_SIZE} samples")
    print(f"🔧 Batch size: {batch_size}, Max tokens: {max_new_tokens}")
else:
    print("🚀 PRODUCTION MODE - Using full cluster settings")
    device = "auto"
    TEST_SUBSET_SIZE = None

⚠️  TEST MODE ENABLED - Using local settings
✓ Using Apple Silicon GPU (MPS)
📊 Will process only 5 samples
🔧 Batch size: 2, Max tokens: 50


In [4]:
# Cell 3: Load dataset
df = pl.read_csv(dataset_path)

# Apply test subset if in test mode
if TEST_MODE and TEST_SUBSET_SIZE:
    df = df.head(TEST_SUBSET_SIZE)
    print(f"📊 TEST MODE: Limited to {len(df)} rows")
else:
    print(f"Loaded {len(df)} rows")

print(f"Columns: {df.columns}")
df.head()

📊 TEST MODE: Limited to 5 rows
Columns: ['', 'Verse', 'Meter', 'char_count']


,Verse,Meter,char_count
i64,str,str,i64
0,"""ah why this boding start this …","""iambic""",6
1,"""that wings my pulse and shoots…","""iambic""",6
2,"""what mean regardless of yon mi…","""iambic""",6
3,"""these earthborn visions sadden…","""iambic""",6
4,"""what strange disorder prompts …","""iambic""",6


In [ ]:
# Cell 4: Option A - Load adapter with transformers (no merge)
if not use_vllm:
    print("Loading model with adapter (transformers)...")
    
    # Load adapter to get base model name
    from huggingface_hub import hf_hub_download
    import json
    
    config_path = hf_hub_download(adapter_model_id, "adapter_config.json")
    with open(config_path) as f:
        adapter_config = json.load(f)
        base_model_name = adapter_config.get("base_model_name_or_path")
    
    print(f"Base model: {base_model_name}")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    # Determine dtype and device based on test mode
    if TEST_MODE:
        # For macOS/CPU testing, use float32 or float16
        if device == "mps":
            dtype = torch.float16  # MPS supports float16
            device_map = {"": device}
        else:
            dtype = torch.float32  # CPU fallback
            device_map = {"": "cpu"}
        print(f"Loading model with dtype={dtype}, device={device}")
    else:
        dtype = torch.bfloat16
        device_map = "auto"
    
    # Load base model
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        torch_dtype=dtype,
        device_map=device_map,
        trust_remote_code=True,
    )
    
    # Load and apply adapter
    model = PeftModel.from_pretrained(model, adapter_model_id)
    model.eval()
    
    print("Model loaded with adapter!")

In [ ]:
# Cell 5: Option B - Merge adapter and use vLLM (faster for large batches)
if use_vllm:
    if not VLLM_AVAILABLE:
        raise ImportError("vLLM not installed. Run: uv pip install vllm")
    
    print("Loading and merging adapter for vLLM...")
    
    # Load adapter config
    from huggingface_hub import hf_hub_download, login
    login(token=os.getenv("HUGGINGFACE_API_KEY"))
    config_path = hf_hub_download(adapter_model_id, "adapter_config.json")
    with open(config_path) as f:
        adapter_config = json.load(f)
        base_model_name = adapter_config.get("base_model_name_or_path")
    
    print(f"Base model: {base_model_name}")
    
    # Load base model
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        torch_dtype=torch.bfloat16,
        device_map="cpu",  # Keep on CPU during merge
        trust_remote_code=True,
    )
    
    # Load and merge adapter
    model = PeftModel.from_pretrained(model, adapter_model_id)
    model = model.merge_and_unload()
    
    # Save merged model temporarily
    merged_path = project_root / "outputs" / "poetic_llm"
    merged_path.mkdir(parents=True, exist_ok=True)
    model.save_pretrained(merged_path)
    
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    tokenizer.save_pretrained(merged_path)
    
    # Clean up transformers model
    del model
    torch.cuda.empty_cache()
    
    # Load with vLLM
    llm = LLM(
        model=str(merged_path),
        dtype="bfloat16",
        trust_remote_code=True,
        max_model_len=2048,
    )
    
    print("vLLM model loaded!")

Loading and merging adapter for vLLM...
Base model: meta-llama/Meta-Llama-3.1-8B-Instruct


`torch_dtype` is deprecated! Use `dtype` instead!


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

SafetensorError: Error while serializing: I/O error: No space left on device (os error 28)

In [ ]:
# Cell 6: Prepare prompts
def format_prompt(text: str, system_prompt: str = "") -> str:
    """Format prompt using the tokenizer's chat template if available."""
    # Try to use the tokenizer's built-in chat template
    if hasattr(tokenizer, "chat_template") and tokenizer.chat_template:
        messages = []
        if system_prompt:
            messages.append({"role": "system", "content": system_prompt})
        messages.append({"role": "user", "content": text})
        
        return tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
    
    # Fallback: simple format for base models or models without chat templates
    if system_prompt:
        return f"{system_prompt}\n\n{text}\n\n"
    return text

# Extract texts from dataset
if text_column not in df.columns:
    raise ValueError(f"Column '{text_column}' not found. Available: {df.columns}")

texts = df[text_column].cast(str).to_list()

# Optional: add a system prompt for instruction-following
system_prompt = ""  # Add your system prompt here if needed
prompts = [format_prompt(text, system_prompt) for text in texts]

print(f"Prepared {len(prompts)} prompts")
print(f"Example prompt:\n{prompts[0][:200]}...")

In [ ]:
# Cell 6b: Verify chat template format
print("Chat template detection:")
if hasattr(tokenizer, "chat_template") and tokenizer.chat_template:
    print("✓ Using tokenizer's built-in chat template (Llama 3.1 format)")
    # Show example
    test_message = [{"role": "user", "content": "Hello"}]
    example = tokenizer.apply_chat_template(test_message, tokenize=False, add_generation_prompt=True)
    print(f"\nExample format:\n{example}")
else:
    print("⚠ No chat template found, using plain text format")

In [ ]:
# Cell 7: Run inference with transformers
if not use_vllm:
    print("Running batch inference with transformers...")
    outputs = []
    
    for i in tqdm(range(0, len(prompts), batch_size)):
        batch = prompts[i:i + batch_size]
        
        # Tokenize
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
        ).to(model.device)
        
        # Generate
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                pad_token_id=tokenizer.pad_token_id,
            )
        
        # Decode
        batch_outputs = tokenizer.batch_decode(
            generated_ids[:, inputs.input_ids.shape[1]:],
            skip_special_tokens=True,
        )
        outputs.extend(batch_outputs)
    
    print(f"Generated {len(outputs)} outputs")

In [ ]:
# Cell 8: Run inference with vLLM
if use_vllm:
    print("Running batch inference with vLLM...")
    
    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_new_tokens,
    )
    
    vllm_outputs = llm.generate(prompts, sampling_params)
    outputs = [output.outputs[0].text for output in vllm_outputs]
    
    print(f"Generated {len(outputs)} outputs")

In [ ]:
# Cell 9: Save results
# Add outputs to dataframe
df_with_outputs = df.with_columns(
    pl.Series(name=output_column, values=outputs)
)

# Save as JSONL
df_with_outputs.write_ndjson(output_path)
print(f"Saved {len(df_with_outputs)} rows to {output_path}")

# Preview
df_with_outputs.head()

In [ ]:
# Cell 10: Preview sample outputs
print("Sample outputs:\n")
for i in range(min(3, len(outputs))):
    print(f"--- Example {i+1} ---")
    print(f"Input: {texts[i][:100]}...")
    print(f"Output: {outputs[i][:200]}...")
    print()